In [256]:
import re
import pickle
from tensorflow.keras.models import load_model

In [257]:
model=load_model('spam_mail_model.h5')

with open('stopwords.pkl','rb') as file:
    all_stopwords=pickle.load(file)

with open('preprocessing.pkl','rb') as file:
    preprocessing=pickle.load(file)

In [258]:
def convert(text):
    text=text.lower()
    text=re.sub(r'[^a-z\s!?]','',text)
    text=re.sub(r'\s+',' ',text)
    sentance=[]
    for word in text.split():
        if word not in all_stopwords:
            sentance.append(word)
    return ' '.join(w for w in sentance)

In [259]:
def lemmatize_text(text):
    text=preprocessing['nlp'](text)
    return ' '.join(word.lemma_ for word in text)

In [260]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def prediction(text):
    preproc=convert(text)
    lemmatization=lemmatize_text(preproc)
    sequence=preprocessing['tokenizer'].texts_to_sequences([lemmatization])
    padding=pad_sequences(sequence,maxlen=preprocessing['max_length'], padding='post', truncating='post')
    predict=model.predict(padding)[0][0]
    return 1 if predict>0.5 else 0

In [261]:
text="""Subject: Invitation for Interview – Software Engineer Position
Body:
Dear Applicant,

Thank you for applying for the Software Engineer position at our company. We are pleased to invite you for an interview.

📅 Date: March 12, 2025
⏰ Time: 10:00 AM
📍 Location: ABC Corp, 123 Street, NY

Please reply to confirm your availability. Looking forward to meeting you!

Best Regards,
HR Team

"""

In [262]:
if prediction(text):
    print("spam")
else:
    print("Ham")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
Ham
